In [1]:
!pip install Flask-Session
!pip install pyngrok
!pip install bcrypt
!pip install gradio_client
!pip install flash


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 5.1 MB/s eta 0:00:00


#ngrok

In [2]:
# Set ngrok authtoken
from pyngrok import ngrok
ngrok.set_auth_token("2nFWeI0Q3JFlCS0U7Oknzu6Jykd_6Bv4xNYRzqTQxvinQigJh")

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image as PILImage
import torchvision.transforms as transforms
import torchvision.models as models
from flask import Flask, request, render_template_string, session
from flask_session import Session
import os
import subprocess
import time
import requests
from werkzeug.utils import secure_filename
import flash

# Function to start ngrok
def start_ngrok():
    # Start the ngrok process with subprocess, specifying that ngrok should tunnel HTTP traffic to port 5000
    ngrok_process = subprocess.Popen(['ngrok', 'http', '5000'])
    # Delay the script for 4 seconds to allow ngrok time to initialize and start the tunnel
    time.sleep(4)
    # Fetch the ngrok tunnel information using an HTTP GET request to ngrok's local API
    response = requests.get('http://localhost:4040/api/tunnels')
    # Parse the JSON response to get the details of the tunnel
    tunnel_info = response.json()
    # Extract the public URL where the ngrok tunnel is accessible
    public_url = tunnel_info['tunnels'][0]['public_url']
    # Print the ngrok tunnel URL to the console
    print(" * ngrok tunnel URL:", public_url)
    # Return the public URL for use elsewhere in the script
    return public_url

In [4]:
import torch
from PIL import Image as PILImage
import torchvision.transforms as transforms
import torchvision.models as models
from flask import Flask, render_template_string, request, redirect, url_for, session
import os
from werkzeug.security import generate_password_hash, check_password_hash
import re
import subprocess
import requests
import time
import uuid
import bcrypt
from flask import Flask, render_template_string, request, redirect, url_for, session
import os
import sqlite3
from flask import Flask, jsonify

In [5]:
import bcrypt

def hash_password(password):
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

def verify_password(stored_password, provided_password):
    return bcrypt.checkpw(provided_password.encode('utf-8'), stored_password.encode('utf-8'))



#Security Templates

## Home page

In [6]:
welcome_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Meal Plan</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            background-color: #fdf9f3;
        }
        .container {
            height: 100vh;
            display: flex;
            justify-content: center;
            align-items: center;
            flex-direction: column;
        }
        .logo img {
            width: 200px;
        }
        .title {
            font-family: 'Segoe Script', cursive;
            font-size: 4rem;
            color: #333;
            font-weight: bold;
            margin-top: 2rem;
        }
        .btn-get-started {
            background-color: #f3c678;
            border: none;
            padding: 10px 20px;
            font-size: 1.2rem;
            color: #ffffff;
            border-radius: 5px;
            margin-bottom: 1rem;
        }
        .btn-get-started:hover {
            background-color: #e0b16e;
        }
        .login-link {
            font-size: 1rem;
            color: #6c757d;
            text-decoration: none;
        }
        .login-link:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div class="container text-center">
        <div class="logo">
            <img src="static/uploads/mealplan_logo.png" alt="MealPlan Logo">
        </div>
        <div class="title">
            MealPlan
        </div>
        <form action="/register" method="get">
            <button type="submit" class="btn btn-get-started">Get Started</button>
        </form>
        <a href="/login" class="login-link">Already a member? Log in</a>
    </div>
</body>
</html>
'''


## Registration page

In [7]:
register_template = '''
<!DOCTYPE html>
<html>
<head>
    <title>Register</title>
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            font-family: Arial, sans-serif;
            background-color: #fdf9f3;
        }
        .form-container {
            background: #ffffff;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 10px rgba(0, 0, 0, 0.2);
            text-align: center;
            width: 400px;
        }
        .form-container h2 {
            margin-bottom: 20px;
        }
        .form-container label {
            display: block;
            margin: 10px 0 5px;
        }
        .form-container input {
            width: 90%;
            padding: 10px;
            margin-bottom: 10px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        .form-container button {
            background-color: #f3c678;
            color: white;
            padding: 10px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            width: 100%;
        }
        .form-container button:hover {
            background-color: #e0b16e;
        }
    </style>
</head>
<body>
    <div class="form-container">
        <h2>Register</h2>
        <form method="post">
            <label>Name:</label>
            <input type="text" name="name" required>
            <label>Email:</label>
            <input type="email" name="email" required>
            <label>Password:</label>
            <input type="password" name="password" required>
            <button type="submit">Register</button>
        </form>
        <br>
        <a href="{{ url_for('login') }}">Already have an account? Login here.</a>
    </div>
</body>
</html>
'''

In [8]:
register_template_error = '''
<!DOCTYPE html>
<html>
<head>
    <title>Register</title>
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            font-family: Arial, sans-serif;
            background-color: #fdf9f3;
        }
        .form-container {
            background: #ffffff;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 10px rgba(0, 0, 0, 0.2);
            text-align: center;
            width: 400px;
        }
        .form-container h2 {
            margin-bottom: 20px;
        }
        .form-container label {
            display: block;
            margin: 10px 0 5px;
        }
        .form-container input {
            width: 90%;
            padding: 10px;
            margin-bottom: 10px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        .form-container button {
            background-color: #f3c678;
            color: white;
            padding: 10px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            width: 100%;
        }
        .form-container button:hover {
            background-color: #e0b16e;
        }
        .error-message {
            color: red;
            background-color: #f8d7da;
            padding: 10px;
            margin-bottom: 10px;
            border: 1px solid #f5c6cb;
            border-radius: 5px;
            text-align: center;
        }
    </style>
</head>
<body>
    <div class="form-container">
        <h2>Register</h2>
        <!-- Error message section (only shown if error exists) -->
        {% if error %}
            <div class="error-message">
                {{ error }}
            </div>
        {% endif %}
        <form method="post">
            <label>Name:</label>
            <input type="text" name="name" required>
            <label>Email:</label>
            <input type="email" name="email" required>
            <label>Password:</label>
            <input type="password" name="password" required>
            <button type="submit">Register</button>
        </form>
        <br>
        <a href="{{ url_for('login') }}">Already have an account? Login here.</a>
    </div>
</body>
</html>
'''


## Login page

In [9]:
login_template = '''
<!DOCTYPE html>
<html>
<head>
    <title>Login</title>
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            font-family: Arial, sans-serif;
            background-color: #fdf9f3;
        }
        .form-container {
            background: #ffffff;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 10px rgba(0, 0, 0, 0.2);
            text-align: center;
            width: 400px;
        }
        .form-container h2 {
            margin-bottom: 20px;
        }
        .form-container label {
            display: block;
            margin: 10px 0 5px;
        }
        .form-container input {
            width: 90%;
            padding: 10px;
            margin-bottom: 10px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        .form-container button {
            background-color: #f3c678;
            color: white;
            padding: 10px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            width: 100%;
        }
        .form-container button:hover {
            background-color: #e0b16e;
        }
        .get-started {
            background-color: yellow;
            color: white;
            padding: 10px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            width: 100%;
            margin-top: 10px;
        }
    </style>
</head>
<body>
    <div class="form-container">
        <h2>Login</h2>
        <form method="post">
            <label>Email:</label>
            <input type="email" name="email" required>
            <label>Password:</label>
            <input type="password" name="password" required>
            <button type="submit">Login</button>
        </form>
        <button class="get-started" onclick="location.href='{{ url_for('register') }}'">Get Started</button>
    </div>
</body>
</html>
'''

In [10]:
login_template_error = '''
<!DOCTYPE html>
<html>
<head>
    <title>Login</title>
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            font-family: Arial, sans-serif;
            background-color: #fdf9f3;
        }
        .form-container {
            background: #ffffff;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 10px rgba(0, 0, 0, 0.2);
            text-align: center;
            width: 400px;
        }
        .form-container h2 {
            margin-bottom: 20px;
        }
        .form-container label {
            display: block;
            margin: 10px 0 5px;
        }
        .form-container input {
            width: 90%;
            padding: 10px;
            margin-bottom: 10px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        .form-container button {
            background-color: #f3c678;
            color: white;
            padding: 10px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            width: 100%;
        }
        .form-container button:hover {
            background-color: #e0b16e;
        }
        .get-started {
            background-color: yellow;
            color: white;
            padding: 10px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            width: 100%;
            margin-top: 10px;
        }
        .error-message {
            color: red;
            background-color: #f8d7da;
            padding: 10px;
            margin-bottom: 10px;
            border: 1px solid #f5c6cb;
            border-radius: 5px;
            text-align: center;
        }
    </style>
</head>
<body>
    <div class="form-container">
        <h2>Login</h2>
        <!-- Error message section (only shown if error exists) -->
        {% if error %}
            <div class="error-message">
                {{ error }}
            </div>
        {% endif %}
        <form method="post">
            <label>Email:</label>
            <input type="email" name="email" required>
            <label>Password:</label>
            <input type="password" name="password" required>
            <button type="submit">Login</button>
        </form>
        <br>
        <button class="get-started" onclick="location.href='{{ url_for('register') }}'">Get Started</button>
    </div>
</body>
</html>
'''


#User Info Templates

## Details page

In [11]:
details_template= '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Enter your details</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            font-family: Arial, sans-serif;
            background-color: #fdf9f3;
        }
        .form-container {
            background: #ffffff;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 10px rgba(0, 0, 0, 0.2);
            text-align: center;
            width: 305px;
        }
        .form-container h2 {
            margin-bottom: 20px;
        }
        .form-container label {
            display: block;
            margin: 10px 0 5px;
        }
        .form-container input {
            width: 100%;
            padding: 10px;
            margin-bottom: 10px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        .form-container button {
            background-color: #f3c678;
            color: white;
            padding: 10px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            width: 100%;
        }
        .form-container select {
            text-align: center;
        }
        .form-container button:hover {
            background-color: #e0b16e;
        }
        .error-message {
            color: red;
            background-color: #f8d7da;
            padding: 10px;
            margin-bottom: 10px;
            border: 1px solid #f5c6cb;
            border-radius: 5px;
            text-align: center;
        }
    </style>
</head>
<body>
    <div class="form-container">
        <h2>Enter Your Details</h2>
        <form method="POST">
            <label>Height(cm):</label>
            <input type="number" name="height" required>
            <label>Weight(kg):</label>
            <input type="number" name="weight" required>
            <label for="goal">Goal:</label>
            <select name="goal" class="form-select" id="goal">
                <option value="Eat healthier">Eat healthier</option>
                <option value="Weight loss">Weight Loss</option>
                <option value="New diet">New Diet</option>
                <option value="Weight gain">Weight Gain</option>
            </select>
            <label>Age:</label>
            <input type="number" name="age" required>
            <label>Weekly Grocery Budget($):</label>
            <input type="number" name="budget" required>
            <label>Allergies:</label>
            <input type="text" name="allergies" required>
            <button type="submit">Submit</button>
        </form>
    </div>
</body>
</html>
'''

## Diet page

In [12]:
diet_form_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Pick Your Diet</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            background-color: #fdf9f3;
        }
        .container {
            margin-top: 5%;
        }
        .option-container {
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 15px;
            margin: 10px 0;
            background-color: #eaeaea;
            border: 2px solid #d6d6d6;
            border-radius: 10px;
            font-size: 1.25rem;
            font-weight: bold;
            color: #000000;
        }
        .form-check-input {
            margin-right: 15px;
        }
        .btn-continue {
            background-color: #f3c678;
            border: none;
            padding: 10px 20px;
            color: #ffffff;
            border-radius: 5px;
        }
        .btn-continue:hover {
            background-color: #e0b16e;
        }
    </style>
</head>
<body class="bg-light">
    <div class="container text-center">
        <h2 class="fw-bold mb-4">Pick Your Diet</h2>
        <form method="POST" action="/diet">
            <div class="form-check option-container">
                <input class="form-check-input" type="radio" name="diet" value="Classic" id="classic" required>
                <label class="form-check-label" for="classic">Classic</label>
            </div>
            <div class="form-check option-container">
                <input class="form-check-input" type="radio" name="diet" value="Low Carb" id="lowcarb">
                <label class="form-check-label" for="lowcarb">Low Carb</label>
            </div>
            <div class="form-check option-container">
                <input class="form-check-input" type="radio" name="diet" value="Vegetarian" id="vegetarian">
                <label class="form-check-label" for="vegetarian">Vegetarian</label>
            </div>
            <div class="form-check option-container">
                <input class="form-check-input" type="radio" name="diet" value="Pescatarian" id="pescatarian">
                <label class="form-check-label" for="pescatarian">Pescatarian</label>
            </div>
            <div class="form-check option-container">
                <input class="form-check-input" type="radio" name="diet" value="Vegan" id="vegan">
                <label class="form-check-label" for="vegan">Vegan</label>
            </div>
            <button type="submit" class="btn btn-continue mt-4">Continue</button>
        </form>
    </div>
</body>
</html>
'''


## Activity page

In [13]:
activity_form_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Activity Level</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            background-color: #fdf9f3;
        }
        .container {
            margin-top: 5%;
        }
        .option-container {
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 15px;
            margin: 10px 0;
            background-color: #eaeaea;
            border: 2px solid #d6d6d6;
            border-radius: 10px;
            font-size: 1.25rem;
            font-weight: bold;
            color: #000000;
        }
        .form-check-input {
            margin-right: 15px;
        }
        .btn-continue {
            background-color: #f3c678;
            border: none;
            padding: 10px 20px;
            color: #ffffff;
            border-radius: 5px;
        }
        .btn-continue:hover {
            background-color: #e0b16e;
        }
    </style>
</head>
<body class="bg-light">
    <div class="container text-center">
        <h2 class="fw-bold mb-4">Activity Level</h2>
        <form method="POST" action="/activity">
            <div class="form-check option-container">
                <input class="form-check-input" type="radio" name="activity" value="High" id="high" required>
                <label class="form-check-label" for="high">High</label>
            </div>
            <div class="form-check option-container">
                <input class="form-check-input" type="radio" name="activity" value="Medium" id="medium">
                <label class="form-check-label" for="medium">Medium</label>
            </div>
            <div class="form-check option-container">
                <input class="form-check-input" type="radio" name="activity" value="Low" id="low">
                <label class="form-check-label" for="low">Low</label>
            </div>
            <div class="form-check option-container">
                <input class="form-check-input" type="radio" name="activity" value="Inexistent" id="inexistent">
                <label class="form-check-label" for="inexistent">Inexistent</label>
            </div>
            <button type="submit" class="btn btn-continue mt-4">Continue</button>
        </form>
    </div>
</body>
</html>
'''


## Dislikes page

In [14]:
dislikes_form_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Dislikes</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            background-color: #fdf9f3;
        }
        .container {
            margin-top: 3%;
            height: 100vh;
            display: flex;
            flex-direction: column;
            justify-content: space-between;
        }
        h2 {
            font-size: 2.5rem;
        }
        .btn-continue {
            background-color: #f3c678;
            border: none;
            padding: 10px 20px;
            color: #ffffff;
            border-radius: 5px;
        }
        .btn-continue:hover {
            background-color: #e0b16e;
        }
        .checkbox-group {
            text-align: left;
            flex-grow: 1;
            overflow-y: auto;
            margin-bottom: 20px;
            border: 1px solid #ccc;
            border-radius: 5px;
            padding: 10px;
            background: #fff;
        }
        label {
            font-weight: bold;
        }
    </style>
</head>
<body class="bg-light">
    <div class="container text-center">
        <h2 class="fw-bold mb-4">What Foods Do You Dislike?</h2>
        <form method="POST" action="/dislikes">

            <!-- List of commonly disliked foods -->
            <div class="checkbox-group">
                <label class="form-label fw-bold">Select from common dislikes:</label>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Anchovies" id="anchovies">
                    <label class="form-check-label" for="anchovies">Anchovies</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Olives" id="olives">
                    <label class="form-check-label" for="olives">Olives</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Cilantro (Coriander)" id="cilantro">
                    <label class="form-check-label" for="cilantro">Cilantro (Coriander)</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Beets" id="beets">
                    <label class="form-check-label" for="beets">Beets</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Liver" id="liver">
                    <label class="form-check-label" for="liver">Liver</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Blue cheese" id="blue-cheese">
                    <label class="form-check-label" for="blue-cheese">Blue cheese</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Eggplant" id="eggplant">
                    <label class="form-check-label" for="eggplant">Eggplant</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Mushrooms" id="mushrooms">
                    <label class="form-check-label" for="mushrooms">Mushrooms</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Tofu" id="tofu">
                    <label class="form-check-label" for="tofu">Tofu</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Brussels sprouts" id="brussels-sprouts">
                    <label class="form-check-label" for="brussels-sprouts">Brussels sprouts</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Truffle oil" id="truffle-oil">
                    <label class="form-check-label" for="truffle-oil">Truffle oil</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Horseradish" id="horseradish">
                    <label class="form-check-label" for="horseradish">Horseradish</label>
                </div>
                <!-- Add more items from the provided list -->
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Marzipan" id="marzipan">
                    <label class="form-check-label" for="marzipan">Marzipan</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Mustard" id="mustard">
                    <label class="form-check-label" for="mustard">Mustard</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Mayonnaise" id="mayonnaise">
                    <label class="form-check-label" for="mayonnaise">Mayonnaise</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Pickles" id="pickles">
                    <label class="form-check-label" for="pickles">Pickles</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Sauerkraut" id="sauerkraut">
                    <label class="form-check-label" for="sauerkraut">Sauerkraut</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Sweetbreads" id="sweetbreads">
                    <label class="form-check-label" for="sweetbreads">Sweetbreads</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Miso" id="miso">
                    <label class="form-check-label" for="miso">Miso</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Seaweed (Nori, Wakame)" id="seaweed">
                    <label class="form-check-label" for="seaweed">Seaweed (Nori, Wakame)</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Okra" id="okra">
                    <label class="form-check-label" for="okra">Okra</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Artichokes" id="artichokes">
                    <label class="form-check-label" for="artichokes">Artichokes</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Cabbage" id="cabbage">
                    <label class="form-check-label" for="cabbage">Cabbage</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Coconut (in certain forms)" id="coconut">
                    <label class="form-check-label" for="coconut">Coconut (in certain forms)</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Sriracha sauce" id="sriracha">
                    <label class="form-check-label" for="sriracha">Sriracha sauce</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Gorgonzola" id="gorgonzola">
                    <label class="form-check-label" for="gorgonzola">Gorgonzola</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Kimchi" id="kimchi">
                    <label class="form-check-label" for="kimchi">Kimchi</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Durian" id="durian">
                    <label class="form-check-label" for="durian">Durian</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Herring" id="herring">
                    <label class="form-check-label" for="herring">Herring</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Pâté" id="pate">
                    <label class="form-check-label" for="pate">Pâté</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Taramasalata" id="taramasalata">
                    <label class="form-check-label" for="taramasalata">Taramasalata</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Mackerel" id="mackerel">
                    <label class="form-check-label" for="mackerel">Mackerel</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Spam" id="spam">
                    <label class="form-check-label" for="spam">Spam</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Vegemite" id="vegemite">
                    <label class="form-check-label" for="vegemite">Vegemite</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Black licorice" id="black-licorice">
                    <label class="form-check-label" for="black-licorice">Black licorice</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Caviar" id="caviar">
                    <label class="form-check-label" for="caviar">Caviar</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Wobbly gelatin desserts" id="gelatin-desserts">
                    <label class="form-check-label" for="gelatin-desserts">Wobbly gelatin desserts</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Liverwurst" id="liverwurst">
                    <label class="form-check-label" for="liverwurst">Liverwurst</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Frogs' legs" id="frogs-legs">
                    <label class="form-check-label" for="frogs-legs">Frogs' legs</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Escargot" id="escargot">
                    <label class="form-check-label" for="escargot">Escargot</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Wasabi" id="wasabi">
                    <label class="form-check-label" for="wasabi">Wasabi</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Curry leaves" id="curry-leaves">
                    <label class="form-check-label" for="curry-leaves">Curry leaves</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Kale" id="kale">
                    <label class="form-check-label" for="kale">Kale</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Fennel" id="fennel">
                    <label class="form-check-label" for="fennel">Fennel</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Radishes" id="radishes">
                    <label class="form-check-label" for="radishes">Radishes</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Anchovy paste" id="anchovy-paste">
                    <label class="form-check-label" for="anchovy-paste">Anchovy paste</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Pumpernickel bread" id="pumpernickel-bread">
                    <label class="form-check-label" for="pumpernickel-bread">Pumpernickel bread</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Chia seeds" id="chia-seeds">
                    <label class="form-check-label" for="chia-seeds">Chia seeds</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Cucumber (in certain preparations)" id="cucumber">
                    <label class="form-check-label" for="cucumber">Cucumber (in certain preparations)</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Tofu skin (yuba)" id="tofu-skin">
                    <label class="form-check-label" for="tofu-skin">Tofu skin (yuba)</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Sweet corn (in certain dishes)" id="sweet-corn">
                    <label class="form-check-label" for="sweet-corn">Sweet corn (in certain dishes)</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Mung beans" id="mung-beans">
                    <label class="form-check-label" for="mung-beans">Mung beans</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Quinoa (for some people, due to texture)" id="quinoa">
                    <label class="form-check-label" for="quinoa">Quinoa (for some people, due to texture)</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Black-eyed peas" id="black-eyed-peas">
                    <label class="form-check-label" for="black-eyed-peas">Black-eyed peas</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Hummus" id="hummus">
                    <label class="form-check-label" for="hummus">Hummus</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Cold cuts" id="cold-cuts">
                    <label class="form-check-label" for="cold-cuts">Cold cuts</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Provolone cheese" id="provolone-cheese">
                    <label class="form-check-label" for="provolone-cheese">Provolone cheese</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Ricotta cheese" id="ricotta-cheese">
                    <label class="form-check-label" for="ricotta-cheese">Ricotta cheese</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Taramasalata" id="taramasalata">
                    <label class="form-check-label" for="taramasalata">Taramasalata</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Mustard greens" id="mustard-greens">
                    <label class="form-check-label" for="mustard-greens">Mustard greens</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Beet greens" id="beet-greens">
                    <label class="form-check-label" for="beet-greens">Beet greens</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Tofu burgers" id="tofu-burgers">
                    <label class="form-check-label" for="tofu-burgers">Tofu burgers</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Tempeh" id="tempeh">
                    <label class="form-check-label" for="tempeh">Tempeh</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Poppy seeds" id="poppy-seeds">
                    <label class="form-check-label" for="poppy-seeds">Poppy seeds</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Oysters" id="oysters">
                    <label class="form-check-label" for="oysters">Oysters</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Sea cucumbers" id="sea-cucumbers">
                    <label class="form-check-label" for="sea-cucumbers">Sea cucumbers</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Prawns (for some)" id="prawns">
                    <label class="form-check-label" for="prawns">Prawns (for some)</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="dislikes" value="Sautéed onions (for certain people)" id="sauteed-onions">
                    <label class="form-check-label" for="sauteed-onions">Sautéed onions (for certain people)</label>
                </div>
            </div>
            <button type="submit" class="btn btn-continue mt-4">Continue</button>
        </form>
    </div>
</body>
</html>


'''


## Preferences page

In [15]:
preferences_form_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Preferences</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            background-color: #fdf9f3;
        }
        .container {
            margin-top: 3%;
        }
        h2 {
            font-size: 2.5rem;
        }
        .btn-continue {
            background-color: #f3c678;
            border: none;
            padding: 10px 20px;
            color: #ffffff;
            border-radius: 5px;
        }
        .btn-continue:hover {
            background-color: #e0b16e;
        }
        .checkbox-group {
            text-align: left;
            max-height: 300px;
            overflow-y: auto;
            margin-bottom: 20px;
            border: 1px solid #ccc;
            border-radius: 5px;
            padding: 10px;
            background: #fff;
        }
        label {
            font-weight: bold;
        }
    </style>
</head>
<body class="bg-light">
    <div class="container text-center">
        <h2 class="fw-bold mb-4">What Are Your Preferences?</h2>
        <form method="POST" action="/preferences">
            <!-- Input field for custom preferences -->
            <label class="form-label fw-bold">Add your own:</label>
            <input type="text" name="custom_preferences" class="form-control mb-3" placeholder="Type ingredients">

            <!-- List of food preferences -->
            <div class="checkbox-group">
                <label class="form-label fw-bold">Select from food preferences:</label>

                <!-- List of food items -->
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Pasta" id="pasta">
                    <label class="form-check-label" for="pasta">Pasta</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="French Fries" id="french-fries">
                    <label class="form-check-label" for="french-fries">French Fries</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Ice Cream" id="ice-cream">
                    <label class="form-check-label" for="ice-cream">Ice Cream</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Bread" id="bread">
                    <label class="form-check-label" for="bread">Bread</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Fried Rice" id="fried-rice">
                    <label class="form-check-label" for="fried-rice">Fried Rice</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Pancakes" id="pancakes">
                    <label class="form-check-label" for="pancakes">Pancakes</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Burger" id="burger">
                    <label class="form-check-label" for="burger">Burger</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Pizza" id="pizza">
                    <label class="form-check-label" for="pizza">Pizza</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Pumpkin Pie" id="pumpkin-pie">
                    <label class="form-check-label" for="pumpkin-pie">Pumpkin Pie</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Chicken Pot Pie" id="chicken-pot-pie">
                    <label class="form-check-label" for="chicken-pot-pie">Chicken Pot Pie</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Banana" id="banana">
                    <label class="form-check-label" for="banana">Banana</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Apple Pie" id="apple-pie">
                    <label class="form-check-label" for="apple-pie">Apple Pie</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Bagel" id="bagel">
                    <label class="form-check-label" for="bagel">Bagel</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Muffins" id="muffins">
                    <label class="form-check-label" for="muffins">Muffins</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Alfredo Sauce" id="alfredo-sauce">
                    <label class="form-check-label" for="alfredo-sauce">Alfredo Sauce</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Reece's Peanut Cups" id="reeses-peanut-cups">
                    <label class="form-check-label" for="reeses-peanut-cups">Reece's Peanut Cups</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Ice Cream Cake" id="ice-cream-cake">
                    <label class="form-check-label" for="ice-cream-cake">Ice Cream Cake</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Cheesecake" id="cheesecake">
                    <label class="form-check-label" for="cheesecake">Cheesecake</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Cheese" id="cheese">
                    <label class="form-check-label" for="cheese">Cheese</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Banana Bread" id="banana-bread">
                    <label class="form-check-label" for="banana-bread">Banana Bread</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Potato Chips" id="potato-chips">
                    <label class="form-check-label" for="potato-chips">Potato Chips</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Cheetos" id="cheetos">
                    <label class="form-check-label" for="cheetos">Cheetos</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Doritos" id="doritos">
                    <label class="form-check-label" for="doritos">Doritos</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Tacos" id="tacos">
                    <label class="form-check-label" for="tacos">Tacos</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Burritos" id="burritos">
                    <label class="form-check-label" for="burritos">Burritos</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Chimichanga" id="chimichanga">
                    <label class="form-check-label" for="chimichanga">Chimichanga</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Enchilada" id="enchilada">
                    <label class="form-check-label" for="enchilada">Enchilada</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Salsa" id="salsa">
                    <label class="form-check-label" for="salsa">Salsa</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Marinara Sauce" id="marinara-sauce">
                    <label class="form-check-label" for="marinara-sauce">Marinara Sauce</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Broccoli" id="broccoli">
                    <label class="form-check-label" for="broccoli">Broccoli</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Chocolate Covered Strawberries" id="chocolate-covered-strawberries">
                    <label class="form-check-label" for="chocolate-covered-strawberries">Chocolate Covered Strawberries</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Kiwi" id="kiwi">
                    <label class="form-check-label" for="kiwi">Kiwi</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Tomato" id="tomato">
                    <label class="form-check-label" for="tomato">Tomato</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Salad" id="salad">
                    <label class="form-check-label" for="salad">Salad</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Steak" id="steak">
                    <label class="form-check-label" for="steak">Steak</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Chicken Tenders" id="chicken-tenders">
                    <label class="form-check-label" for="chicken-tenders">Chicken Tenders</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Grilled Chicken" id="grilled-chicken">
                    <label class="form-check-label" for="grilled-chicken">Grilled Chicken</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Ribs" id="ribs">
                    <label class="form-check-label" for="ribs">Ribs</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Biscuits and Gravy" id="biscuits-and-gravy">
                    <label class="form-check-label" for="biscuits-and-gravy">Biscuits and Gravy</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Hot Dogs" id="hot-dogs">
                    <label class="form-check-label" for="hot-dogs">Hot Dogs</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Fried Chicken" id="fried-chicken">
                    <label class="form-check-label" for="fried-chicken">Fried Chicken</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Roasted Chicken and Garlic" id="roasted-chicken-and-garlic">
                    <label class="form-check-label" for="roasted-chicken-and-garlic">Roasted Chicken and Garlic</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Eggs" id="eggs">
                    <label class="form-check-label" for="eggs">Eggs</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Bacon" id="bacon">
                    <label class="form-check-label" for="bacon">Bacon</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Sausage" id="sausage">
                    <label class="form-check-label" for="sausage">Sausage</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Mashed Potatoes" id="mashed-potatoes">
                    <label class="form-check-label" for="mashed-potatoes">Mashed Potatoes</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Stuffing" id="stuffing">
                    <label class="form-check-label" for="stuffing">Stuffing</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Brownies" id="brownies">
                    <label class="form-check-label" for="brownies">Brownies</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Cookies" id="cookies">
                    <label class="form-check-label" for="cookies">Cookies</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Submarine Sandwiches" id="submarine-sandwiches">
                    <label class="form-check-label" for="submarine-sandwiches">Submarine Sandwiches</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Donuts" id="donuts">
                    <label class="form-check-label" for="donuts">Donuts</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Turkey" id="turkey">
                    <label class="form-check-label" for="turkey">Turkey</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Cranberry" id="cranberry">
                    <label class="form-check-label" for="cranberry">Cranberry</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Gravy" id="gravy">
                    <label class="form-check-label" for="gravy">Gravy</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Green Beans" id="green-beans">
                    <label class="form-check-label" for="green-beans">Green Beans</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Mac and Cheese" id="mac-and-cheese">
                    <label class="form-check-label" for="mac-and-cheese">Mac and Cheese</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Soup" id="soup">
                    <label class="form-check-label" for="soup">Soup</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Chicken Noodle Soup" id="chicken-noodle-soup">
                    <label class="form-check-label" for="chicken-noodle-soup">Chicken Noodle Soup</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Clam Chowder" id="clam-chowder">
                    <label class="form-check-label" for="clam-chowder">Clam Chowder</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Pho" id="pho">
                    <label class="form-check-label" for="pho">Pho</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Chicken Tikka Masala" id="chicken-tikka-masala">
                    <label class="form-check-label" for="chicken-tikka-masala">Chicken Tikka Masala</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Samosa" id="samosa">
                    <label class="form-check-label" for="samosa">Samosa</label>
                </div>
                <div class="form-check">
                    <input class="form-check-input" type="checkbox" name="preferences" value="Naan" id="naan">
                    <label class="form-check-label" for="naan">Naan</label>
                </div>
                <!-- Submit button -->
                <button type="submit" class="btn btn-continue mt-4">Continue</button>
            </div>
            <!-- Submit button -->
            <button type="submit" class="btn btn-continue mt-4">Continue</button>
        </form>
    </div>
</body>
</html>

'''

# Navigation Bar Templates

## Dashboard page

In [16]:
dashboard_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Dashboard</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            margin: 0;
            padding: 0;
            background-color: #fdf9f3;
            font-family: Arial, sans-serif;
        }
        .header {
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 10px 20px;
            background-color: #fdf9f3;
            border-bottom: 1px solid #ddd;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }
        .header-buttons {
            display: flex;
            gap: 10px;
        }
        .button {
            background-color: #f3c678;
            color: white;
            border: none;
            border-radius: 25px;
            padding: 10px 20px;
            font-size: 14px;
            font-weight: bold;
            cursor: pointer;
            transition: background-color 0.3s ease;
            text-decoration: none;
        }
        .button:hover {
            background-color: #e0b16e;
        }
        .button-container {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 80vh; /* Center vertically */
        }
        .meal-plan-button {
            background-color: #f3c678;
            color: white;
            border: none;
            border-radius: 25px;
            padding: 20px;
            font-size: 18px;
            font-weight: bold;
            width: 90%; /* Almost full width */
            max-width: 600px; /* Limit max size */
            cursor: pointer;
            text-align: center;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            transition: background-color 0.3s ease;
        }
        .meal-plan-button:hover {
            background-color: #e0b16e;
        }
        .navbar {
            position: fixed;
            bottom: 0;
            width: 100%;
            background-color: white;
            box-shadow: 0 -2px 5px rgba(0, 0, 0, 0.1);
            display: flex;
            justify-content: space-around;
            padding: 10px 0;
        }
        .navbar-item {
            display: flex;
            flex-direction: column;
            align-items: center;
            font-size: 14px;
            color: gray;
        }
        .navbar-item svg {
            width: 24px;
            height: 24px;
            margin-bottom: 5px;
        }
    </style>
</head>
<body>
    <div class="header">
        <div class="header-buttons">
            <a href="/logout" class="button">Logout</a>
        </div>
    </div>
    <div class="button-container">
        <a href="/diet" class="meal-plan-button">Get Your First Meal Plan</a>
    </div>
    <div class="navbar">
        <a href="/groceries" class="navbar-item">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-cart" viewBox="0 0 16 16">
                <path d="M0 1.5A.5.5 0 0 1 .5 1H2a.5.5 0 0 1 .485.379L2.89 3H14.5a.5.5 0 0 1 .491.592l-1.5 8A.5.5 0 0 1 13 12H4a.5.5 0 0 1-.491-.408L2.01 3.607 1.61 2H.5a.5.5 0 0 1-.5-.5M3.102 4l1.313 7h8.17l1.313-7zM5 12a2 2 0 1 0 0 4 2 2 0 0 0 0-4m7 0a2 2 0 1 0 0 4 2 2 0 0 0 0-4m-7 1a1 1 0 1 1 0 2 1 1 0 0 1 0-2m7 0a1 1 0 1 1 0 2 1 1 0 0 1 0-2"/>
            </svg>
            <span>Groceries</span>
        </a>
        <a href="/meal-plan" class="navbar-item">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-grid-1x2" viewBox="0 0 16 16">
                <path d="M6 1H1v14h5zm9 0h-5v5h5zm0 9v5h-5v-5zM0 1a1 1 0 0 1 1-1h5a1 1 0 0 1 1 1v14a1 1 0 0 1-1 1H1a1 1 0 0 1-1-1zm9 0a1 1 0 0 1 1-1h5a1 1 0 0 1 1 1v5a1 1 0 0 1-1 1h-5a1 1 0 0 1-1-1zm1 8a1 1 0 0 0-1 1v5a1 1 0 0 0 1 1h5a1 1 0 0 0 1-1v-5a1 1 0 0 0-1-1z"/>
            </svg>
            <span>Meal Plan</span>
        </a>
        <a href="/chatbot" class="navbar-item">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-robot" viewBox="0 0 16 16">
                <path d="M6 12.5a.5.5 0 0 1 .5-.5h3a.5.5 0 0 1 0 1h-3a.5.5 0 0 1-.5-.5M3 8.062C3 6.76 4.235 5.765 5.53 5.886a26.6 26.6 0 0 0 4.94 0C11.765 5.765 13 6.76 13 8.062v1.157a.93.93 0 0 1-.765.935c-.845.147-2.34.346-4.235.346s-3.39-.2-4.235-.346A.93.93 0 0 1 3 9.219zM4.542 7.235a.25.25 0 0 0-.217.068l-.92.9a25 25 0 0 1-1.871-.183.25.25 0 0 0-.068.495c.55.076 1.232.149 2.02.193a.25.25 0 0 0 .189-.071l.754-.736.847 1.71a.25.25 0 0 0 .404.062l.932-.97a25 25 0 0 0 1.922-.188.25.25 0 0 0-.068-.495c-.538.074-1.207.145-1.98.189a.25.25 0 0 0-.166.076l-.754.785-.842-1.7a.25.25 0 0 0-.182-.135"/>
                <path d="M8.5 1.866a1 1 0 1 0-1 0V3h-2A4.5 4.5 0 0 0 1 7.5V8a1 1 0 0 0-1 1v2a1 1 0 0 0 1 1v1a2 2 0 0 0 2 2h10a2 2 0 0 0 2-2v-1a1 1 0 0 0 1-1V9a1 1 0 0 0-1-1v-.5A4.5 4.5 0 0 0 10.5 3h-2zM14 7.5V13a1 1 0 0 1-1 1H3a1 1 0 0 1-1-1V7.5A3.5 3.5 0 0 1 5.5 4h5A3.5 3.5 0 0 1 14 7.5"/>
            </svg>
            <span>MealBot</span>
        </a>
        <a href="/profile" class="navbar-item">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-person" viewBox="0 0 16 16">
                <path d="M8 8a3 3 0 1 0 0-6 3 3 0 0 0 0 6m2-3a2 2 0 1 1-4 0 2 2 0 0 1 4 0zM1 14s1-4 7-4 7 4 7 4H1zM2.708 13h10.584c-.15-.375-.509-.937-1.192-1.451C11.58 10.926 9.685 10 8 10c-1.685 0-3.58.926-4.1 1.549-.683.514-1.042 1.076-1.192 1.451"/>
            </svg>
            <span>Profile</span>
        </a>
    </div>
</body>
</html>
'''


## Profile page

In [26]:
profile_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Edit Profile</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            background-color: #fdf9f3;
        }
        .back-button {
            position: absolute;
            top: 20px;
            left: 20px;
            background-color: #f3c678;
            color: white;
            border: none;
            border-radius: 5px;
            padding: 10px 15px;
            text-decoration: none;
            font-weight: bold;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }
        .back-button:hover {
            background-color: #e0b16e;
        }
        .container {
            margin: 5% auto;
            max-width: 600px;
            padding: 20px;
            background-color: #ffffff;
            border: 1px solid #e0b16e;
            border-radius: 10px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }
        .profile-header {
            text-align: center;
            font-weight: bold;
            color: #f3c678;
            margin-bottom: 20px;
        }
        .form-group {
            margin-bottom: 15px;
        }
        label {
            font-weight: bold;
            color: #f3c678;
        }
        .btn-save {
            background-color: #f3c678;
            border: none;
            padding: 10px 20px;
            color: #ffffff;
            border-radius: 5px;
            cursor: pointer;
        }
        .btn-save:hover {
            background-color: #e0b16e;
        }
        .alert {
            margin-top: 15px;
        }
    </style>
</head>
<body>
    <a href="{{ back_link }}" class="back-button">← Back to Dashboard</a>

    <div class="container">
        <h2 class="profile-header">Edit Profile</h2>

        <!-- Display errors or success messages -->
        {% if error %}
        <div class="alert alert-danger">{{ error }}</div>
        {% endif %}

        <form method="POST" action="/profile">
            <div class="form-group">
                <label for="name">Name:</label>
                <input type="text" class="form-control" id="name" name="name" value="{{ name }}" required>
            </div>
            <div class="form-group">
                <label for="age">Age:</label>
                <input type="number" class="form-control" id="age" name="age" value="{{ age }}" required>
            </div>
            <div class="form-group">
                <label for="weight">Weight(kg):</label>
                <input type="text" class="form-control" id="weight" name="weight" value="{{ weight }}" required>
            </div>
            <div class="form-group">
                <label for="height">Height(cm):</label>
                <input type="text" class="form-control" id="height" name="height" value="{{ height }}" required>
            </div>
            <div class="form-group">
                <label for="goal">Goal:</label>
                <select name="goal" class="form-select" id="goal">
                    <option value="Eat healthier" {% if goal == "Eat healthier" %}selected{% endif %}>Eat healthier</option>
                    <option value="Weight loss" {% if goal == "Weight loss" %}selected{% endif %}>Weight Loss</option>
                    <option value="New diet" {% if goal == "New diet" %}selected{% endif %}>New Diet</option>
                    <option value="Weight gain" {% if goal == "Weight gain" %}selected{% endif %}>Weight Gain</option>
                </select>
            </div>
            <div class="form-group">
                <label for="budget">Budget($):</label>
                <input type="text" class="form-control" id="budget" name="budget" value="{{ budget }}" required>
            </div>
            <div class="form-group">
                <label for="allergies">Allergies:</label>
                <input type="text" class="form-control" id="allergies" name="allergies" value="{{ allergies }}">
            </div>
            <button type="submit" class="btn btn-save">Save Changes</button>
        </form>
    </div>
</body>
</html>
'''

## Groceries page

In [18]:
groceries_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Groceries</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            background-color: #fdf9f3;
            margin: 0;
            font-family: Arial, sans-serif;
        }
        .container {
            margin: 5% auto;
            max-width: 800px;
            background-color: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }
        .header {
            text-align: center;
            font-weight: bold;
            color: #f3c678;
            margin-bottom: 20px;
            font-size: 24px;
        }
        .search-bar {
            margin-bottom: 20px;
        }
        .search-bar input {
            width: 100%;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 5px;
            font-size: 16px;
        }
        .ingredients-list {
            max-height: 400px;
            overflow-y: auto;
            border: 1px solid #ddd;
            border-radius: 5px;
            padding: 10px;
        }
        .ingredients-list label {
            display: block;
            margin: 5px 0;
        }
        .footer {
            position: fixed;
            bottom: 0;
            width: 100%;
            background-color: #fff;
            box-shadow: 0 -2px 5px rgba(0, 0, 0, 0.1);
            display: flex;
            justify-content: space-around;
            padding: 10px 0;
        }
        .footer a {
            text-decoration: none;
            color: gray;
            font-size: 14px;
            text-align: center;
        }
        .footer svg {
            width: 24px;
            height: 24px;
        }
    </style>
    <script>
        function filterIngredients() {
            let input = document.getElementById('search-input').value.toLowerCase();
            let labels = document.querySelectorAll('.ingredients-list label');
            labels.forEach(label => {
                if (label.textContent.toLowerCase().includes(input)) {
                    label.style.display = "block";
                } else {
                    label.style.display = "none";
                }
            });
        }
    </script>
</head>
<body>
    <div class="container">
        <h2 class="header">Groceries</h2>
        <div class="search-bar">
            <input id="search-input" type="text" placeholder="Type ingredients" onkeyup="filterIngredients()">
        </div>
        <form method="POST" action="/groceries">
            <div class="ingredients-list">
                <!-- Show selected ingredients first -->
                {% if selected_ingredients %}
                <h5>Selected Ingredients:</h5>
                {% for ingredient in selected_ingredients %}
                <label>
                    <input type="checkbox" name="groceries" value="{{ ingredient }}" checked> {{ ingredient }}
                </label>
                {% endfor %}
                {% endif %}
                <h5>Other Ingredients:</h5>
                {% for ingredient in remaining_ingredients %}
                <label>
                    <input type="checkbox" name="groceries" value="{{ ingredient }}"> {{ ingredient }}
                </label>
                {% endfor %}
            </div>
            <button type="submit" class="btn mt-3 w-100" style="background-color: #f3c678; color: white; border: none; font-size: 18px;">
                Save Selected Ingredients
            </button>
        </form>
    </div>
    <div class="footer">
        <a href="/groceries">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-cart" viewBox="0 0 16 16">
                <path d="M0 1.5A.5.5 0 0 1 .5 1H2a.5.5 0 0 1 .485.379L2.89 3H14.5a.5.5 0 0 1 .491.592l-1.5 8A.5.5 0 0 1 13 12H4a.5.5 0 0 1-.491-.408L2.01 3.607 1.61 2H.5a.5.5 0 0 1-.5-.5M3.102 4l1.313 7h8.17l1.313-7zM5 12a2 2 0 1 0 0 4 2 2 0 0 0 0-4m7 0a2 2 0 1 0 0 4 2 2 0 0 0 0-4m-7 1a1 1 0 1 1 0 2 1 1 0 0 1 0-2m7 0a1 1 0 1 1 0 2 1 1 0 0 1 0-2"/>
            </svg>
            Groceries
        </a>
        <a href="/meal-plan">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-grid-1x2" viewBox="0 0 16 16">
                <path d="M6 1H1v14h5zm9 0h-5v5h5zm0 9v5h-5v-5zM0 1a1 1 0 0 1 1-1h5a1 1 0 0 1 1 1v14a1 1 0 0 1-1 1H1a1 1 0 0 1-1-1zm9 0a1 1 0 0 1 1-1h5a1 1 0 0 1 1 1v5a1 1 0 0 1-1 1h-5a1 1 0 0 1-1-1zm1 8a1 1 0 0 0-1 1v5a1 1 0 0 0 1 1h5a1 1 0 0 0 1-1v-5a1 1 0 0 0-1-1z"/>
            </svg>
            Meal Plan
        </a>
        <a href="/chatbot">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-robot" viewBox="0 0 16 16">
                <path d="M8 1.866a1 1 0 1 0-1 0V3h-2A4.5 4.5 0 0 0 1 7.5V8a1 1 0 0 0-1 1v2a1 1 0 0 0 1 1v1a2 2 0 0 0 2 2h10a2 2 0 0 0 2-2v-1a1 1 0 0 0 1-1V9a1 1 0 0 0-1-1v-.5A4.5 4.5 0 0 0 10.5 3h-2zM14 7.5V13a1 1 0 0 1-1 1H3a1 1 0 0 1-1-1V7.5A3.5 3.5 0 0 1 5.5 4h5A3.5 3.5 0 0 1 14 7.5"/>
            </svg>
            ChatBot
        </a>
        <a href="/profile">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-person" viewBox="0 0 16 16">
                <path d="M8 8a3 3 0 1 0 0-6 3 3 0 0 0 0 6m2-3a2 2 0 1 1-4 0 2 2 0 0 1 4 0zM1 14s1-4 7-4 7 4 7 4H1zM2.708 13h10.584c-.15-.375-.509-.937-1.192-1.451C11.58 10.926 9.685 10 8 10c-1.685 0-3.58.926-4.1 1.549-.683.514-1.042 1.076-1.192 1.451"/>
            </svg>
            Profile
        </a>
    </div>
</body>
</html>
'''

## Meal plan page

In [19]:
meal_plan_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Meal Plan Dashboard</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            margin: 0;
            padding: 0;
            background-color: #fdf9f3;
            font-family: Arial, sans-serif;
        }
        .header {
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 10px 20px;
            background-color: #fdf9f3;
            border-bottom: 1px solid #ddd;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }
        .header-buttons {
            display: flex;
            gap: 10px;
        }
        .button {
            background-color: #f3c678;
            color: white;
            border: none;
            border-radius: 25px;
            padding: 10px 20px;
            font-size: 14px;
            font-weight: bold;
            cursor: pointer;
            transition: background-color 0.3s ease;
            text-decoration: none;
        }
        .button:hover {
            background-color: #e0b16e;
        }
        .content {
            padding: 20px;
        }
        .day-section {
            margin-bottom: 20px;
            padding: 15px;
            background-color: #ffffff;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }
        .day-section h5 {
            margin-top: 0;
            margin-bottom: 10px;
            font-size: 18px;
            color: #333;
        }
        .day-section p {
            margin: 5px 0;
            color: #555;
        }
        .navbar {
            position: fixed;
            bottom: 0;
            width: 100%;
            background-color: white;
            box-shadow: 0 -2px 5px rgba(0, 0, 0, 0.1);
            display: flex;
            justify-content: space-around;
            padding: 10px 0;
        }
        .navbar-item {
            display: flex;
            flex-direction: column;
            align-items: center;
            font-size: 14px;
            color: gray;
        }
        .navbar-item svg {
            width: 24px;
            height: 24px;
            margin-bottom: 5px;
        }
    </style>
</head>
<body>
    <div class="header">
        <h4>Your Meal Plan</h4>
        <div class="header-buttons">
            <a href="/diet" class="button">Create Another Meal Plan</a>
            <a href="/logout" class="button">Logout</a>
        </div>
    </div>
    <div class="content">
        {% for day, meals in week_plan.items() %}
        <div class="day-section">
            <h5>{{ day }}</h5>
            <ul>
                {% for meal in meals %}
                <li>{{ meal }}</li>
                {% endfor %}
            </ul>
        </div>
        {% endfor %}
    </div>
    <div class="navbar">
        <a href="/groceries" class="navbar-item">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-cart" viewBox="0 0 16 16">
                <path d="M0 1.5A.5.5 0 0 1 .5 1H2a.5.5 0 0 1 .485.379L2.89 3H14.5a.5.5 0 0 1 .491.592l-1.5 8A.5.5 0 0 1 13 12H4a.5.5 0 0 1-.491-.408L2.01 3.607 1.61 2H.5a.5.5 0 0 1-.5-.5M3.102 4l1.313 7h8.17l1.313-7zM5 12a2 2 0 1 0 0 4 2 2 0 0 0 0-4m7 0a2 2 0 1 0 0 4 2 2 0 0 0 0-4m-7 1a1 1 0 1 1 0 2 1 1 0 0 1 0-2m7 0a1 1 0 1 1 0 2 1 1 0 0 1 0-2"/>
            </svg>
            <span>Groceries</span>
        </a>
        <a href="/meal-plan" class="navbar-item">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-grid-1x2" viewBox="0 0 16 16">
                <path d="M6 1H1v14h5zm9 0h-5v5h5zm0 9v5h-5v-5zM0 1a1 1 0 0 1 1-1h5a1 1 0 0 1 1 1v14a1 1 0 0 1-1 1H1a1 1 0 0 1-1-1zm9 0a1 1 0 0 1 1-1h5a1 1 0 0 1 1 1v5a1 1 0 0 1-1 1h-5a1 1 0 0 1-1-1zm1 8a1 1 0 0 0-1 1v5a1 1 0 0 0 1 1h5a1 1 0 0 0 1-1v-5a1 1 0 0 0-1-1z"/>
            </svg>
            <span>Meal Plan</span>
        </a>
        <a href="/chatbot" class="navbar-item">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-robot" viewBox="0 0 16 16">
                <path d="M6 12.5a.5.5 0 0 1 .5-.5h3a.5.5 0 0 1 0 1h-3a.5.5 0 0 1-.5-.5M3 8.062C3 6.76 4.235 5.765 5.53 5.886a26.6 26.6 0 0 0 4.94 0C11.765 5.765 13 6.76 13 8.062v1.157a.93.93 0 0 1-.765.935c-.845.147-2.34.346-4.235.346s-3.39-.2-4.235-.346A.93.93 0 0 1 3 9.219zM4.542 7.235a.25.25 0 0 0-.217.068l-.92.9a25 25 0 0 1-1.871-.183.25.25 0 0 0-.068.495c.55.076 1.232.149 2.02.193a.25.25 0 0 0 .189-.071l.754-.736.847 1.71a.25.25 0 0 0 .404.062l.932-.97a25 25 0 0 0 1.922-.188.25.25 0 0 0-.068-.495c-.538.074-1.207.145-1.98.189a.25.25 0 0 0-.166.076l-.754.785-.842-1.7a.25.25 0 0 0-.182-.135"/>
                <path d="M8.5 1.866a1 1 0 1 0-1 0V3h-2A4.5 4.5 0 0 0 1 7.5V8a1 1 0 0 0-1 1v2a1 1 0 0 0 1 1v1a2 2 0 0 0 2 2h10a2 2 0 0 0 2-2v-1a1 1 0 0 0 1-1V9a1 1 0 0 0-1-1v-.5A4.5 4.5 0 0 0 10.5 3h-2zM14 7.5V13a1 1 0 0 1-1 1H3a1 1 0 0 1-1-1V7.5A3.5 3.5 0 0 1 5.5 4h5A3.5 3.5 0 0 1 14 7.5z"/>
            </svg>
            <span>MealBot</span>
        </a>
        <a href="/profile" class="navbar-item">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-person" viewBox="0 0 16 16">
                <path d="M8 8a3 3 0 1 0 0-6 3 3 0 0 0 0 6m2-3a2 2 0 1 1-4 0 2 2 0 0 1 4 0zM1 14s1-4 7-4 7 4 7 4H1zM2.708 13h10.584c-.15-.375-.509-.937-1.192-1.451C11.58 10.926 9.685 10 8 10c-1.685 0-3.58.926-4.1 1.549-.683.514-1.042 1.076-1.192 1.451"/>
            </svg>
            <span>Profile</span>
        </a>
    </div>
</body>
</html>
'''

In [20]:
meal_plan_template_error = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Error!</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>

        body {
            background-color: #fdf9f3;
            margin: 0;
            padding: 0;
            font-family: Arial, sans-serif;
        }
        .header {
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 10px 20px;
            background-color: #fdf9f3;
            border-bottom: 1px solid #ddd;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }
        .header-buttons {
            display: flex;
            gap: 10px;
        }
        .button {
            background-color: #f3c678;
            color: white;
            border: none;
            border-radius: 25px;
            padding: 10px 20px;
            font-size: 14px;
            font-weight: bold;
            cursor: pointer;
            transition: background-color 0.3s ease;
            text-decoration: none;
        }
        .button:hover {
            background-color: #e0b16e;
        }
        .button-container {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 80vh; /* Center vertically */
        }
        .meal-plan-button {
            background-color: #f3c678;
            color: white;
            border: none;
            border-radius: 25px;
            padding: 20px;
            font-size: 18px;
            font-weight: bold;
            width: 90%; /* Almost full width */
            max-width: 600px; /* Limit max size */
            cursor: pointer;
            text-align: center;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            transition: background-color 0.3s ease;
        }
        .meal-plan-button:hover {
            background-color: #e0b16e; /* Darker orange */
        }
        .content {
            padding: 20px;
        }
        .day-section {
            margin-bottom: 20px;
            padding: 15px;
            background-color: #ffffff;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }
        .day-section h5 {
            margin-top: 0;
            margin-bottom: 10px;
            font-size: 18px;
            color: #333;
        }
        .day-section p {
            margin: 5px 0;
            color: #555;
        }
        .navbar {
            position: fixed;
            bottom: 0;
            width: 100%;
            background-color: white;
            box-shadow: 0 -2px 5px rgba(0, 0, 0, 0.1);
            display: flex;
            justify-content: space-around;
            padding: 10px 0;
        }
        .navbar-item {
            display: flex;
            flex-direction: column;
            align-items: center;
            font-size: 14px;
            color: gray;
        }
        .navbar-item svg {
            width: 24px;
            height: 24px;
            margin-bottom: 5px;
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>Error! No meal plan found. Please generate one using the Meal Bot!</h1>
    </div>
    <div class="button-container">
        <a href="/dashboard" class="meal-plan-button">Back to Dashboard</a>
    </div>
    <div class="navbar">
        <a href="/groceries" class="navbar-item">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-cart" viewBox="0 0 16 16">
                <path d="M0 1.5A.5.5 0 0 1 .5 1H2a.5.5 0 0 1 .485.379L2.89 3H14.5a.5.5 0 0 1 .491.592l-1.5 8A.5.5 0 0 1 13 12H4a.5.5 0 0 1-.491-.408L2.01 3.607 1.61 2H.5a.5.5 0 0 1-.5-.5M3.102 4l1.313 7h8.17l1.313-7zM5 12a2 2 0 1 0 0 4 2 2 0 0 0 0-4m7 0a2 2 0 1 0 0 4 2 2 0 0 0 0-4m-7 1a1 1 0 1 1 0 2 1 1 0 0 1 0-2m7 0a1 1 0 1 1 0 2 1 1 0 0 1 0-2"/>
            </svg>
            <span>Groceries</span>
        </a>
        <a href="/meal-plan" class="navbar-item">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-grid-1x2" viewBox="0 0 16 16">
                <path d="M6 1H1v14h5zm9 0h-5v5h5zm0 9v5h-5v-5zM0 1a1 1 0 0 1 1-1h5a1 1 0 0 1 1 1v14a1 1 0 0 1-1 1H1a1 1 0 0 1-1-1zm9 0a1 1 0 0 1 1-1h5a1 1 0 0 1 1 1v5a1 1 0 0 1-1 1h-5a1 1 0 0 1-1-1zm1 8a1 1 0 0 0-1 1v5a1 1 0 0 0 1 1h5a1 1 0 0 0 1-1v-5a1 1 0 0 0-1-1z"/>
            </svg>
            <span>Meal Plan</span>
        </a>
        <a href="/chatbot" class="navbar-item">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-robot" viewBox="0 0 16 16">
                <path d="M6 12.5a.5.5 0 0 1 .5-.5h3a.5.5 0 0 1 0 1h-3a.5.5 0 0 1-.5-.5M3 8.062C3 6.76 4.235 5.765 5.53 5.886a26.6 26.6 0 0 0 4.94 0C11.765 5.765 13 6.76 13 8.062v1.157a.93.93 0 0 1-.765.935c-.845.147-2.34.346-4.235.346s-3.39-.2-4.235-.346A.93.93 0 0 1 3 9.219zM4.542 7.235a.25.25 0 0 0-.217.068l-.92.9a25 25 0 0 1-1.871-.183.25.25 0 0 0-.068.495c.55.076 1.232.149 2.02.193a.25.25 0 0 0 .189-.071l.754-.736.847 1.71a.25.25 0 0 0 .404.062l.932-.97a25 25 0 0 0 1.922-.188.25.25 0 0 0-.068-.495c-.538.074-1.207.145-1.98.189a.25.25 0 0 0-.166.076l-.754.785-.842-1.7a.25.25 0 0 0-.182-.135"/>
                <path d="M8.5 1.866a1 1 0 1 0-1 0V3h-2A4.5 4.5 0 0 0 1 7.5V8a1 1 0 0 0-1 1v2a1 1 0 0 0 1 1v1a2 2 0 0 0 2 2h10a2 2 0 0 0 2-2v-1a1 1 0 0 0 1-1V9a1 1 0 0 0-1-1v-.5A4.5 4.5 0 0 0 10.5 3h-2zM14 7.5V13a1 1 0 0 1-1 1H3a1 1 0 0 1-1-1V7.5A3.5 3.5 0 0 1 5.5 4h5A3.5 3.5 0 0 1 14 7.5z"/>
            </svg>
            <span>MealBot</span>
        </a>
        <a href="/profile" class="navbar-item">
            <svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" class="bi bi-person" viewBox="0 0 16 16">
                <path d="M8 8a3 3 0 1 0 0-6 3 3 0 0 0 0 6m2-3a2 2 0 1 1-4 0 2 2 0 0 1 4 0zM1 14s1-4 7-4 7 4 7 4H1zM2.708 13h10.584c-.15-.375-.509-.937-1.192-1.451C11.58 10.926 9.685 10 8 10c-1.685 0-3.58.926-4.1 1.549-.683.514-1.042 1.076-1.192 1.451"/>
            </svg>
            <span>Profile</span>
        </a>
    </div>
</body>
</html>
'''

## Meal Bot page

In [21]:
chat_template = '''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Meal Planner</title>
        <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f0f0f0;
                margin: 0;
                padding: 0;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                flex-direction: column;
            }

            .container {
                width: 100%;
                max-width: 800px;
                background-color: #ffffff;
                border-radius: 10px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
                display: flex;
                flex-direction: column;
                height: 100%;
            }

            h1 {
                text-align: center;
                padding: 20px;
                color: #333;
                margin: 0;
                background-color: #ffcc80;
                color: white;
                border-radius: 10px 10px 0 0;
            }

            .chat-container {
                display: flex;
                flex-direction: column;
                gap: 10px;
                padding: 20px;
                flex-grow: 1;
                overflow-y: auto;
                padding-bottom: 80px;
            }

            .chat-box {
                display: flex;
                flex-direction: column;
                gap: 10px;
            }

            .message {
                padding: 10px;
                border-radius: 5px;
                max-width: 85%;
            }

            .user-message {
                background-color: #d1f7d6;
                align-self: flex-start;
            }

            .bot-message {
                background-color: #f1f1f1;
                align-self: flex-end;
            }

            .chat-form {
                display: flex;
                gap: 10px;
                padding: 20px;
                background-color: #f9f9f9;
                border-top: 1px solid #ddd;
            }

            .chat-form input {
                width: 100%;
                padding: 10px;
                border-radius: 5px;
                border: 1px solid #ccc;
                font-size: 16px;
            }

            .chat-form button {
                padding: 10px 20px;
                border-radius: 5px;
                border: 1px solid #ccc;
                background-color: #ffcc80;
                color: white;
                cursor: pointer;
                font-size: 16px;
            }

            .chat-form button:hover {
                background-color: #ffb74d;
            }

            .dashboard-button {
                padding: 10px 20px;
                margin: 20px;
                border-radius: 5px;
                border: none;
                background-color: #4CAF50;
                color: white;
                cursor: pointer;
                font-size: 16px;
                text-align: center;
            }

            .dashboard-button:hover {
                background-color: #45a049;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Meal Bot</h1>
            <div class="chat-container">
                <div class="chat-box">
                    {% if user_message %}
                        <div class="message user-message">
                            <p>{{ user_message }}</p>
                        </div>
                    {% endif %}

                    {% if chatbot_response %}
                        <div class="message bot-message">
                            <p>{{ chatbot_response }}</p>
                        </div>
                    {% endif %}
                </div>
            </div>
            <form method="post" class="chat-form">
                <input type="text" id="user_message" name="user_message" placeholder="Type 'Suggest Recipes' " required>
                <button type="submit">Send</button>
            </form>
            <form action="/meal-plan" method="get" style="text-align: center;">
                <button type="submit" class="dashboard-button">View Meal Plan</button>
            </form>
        </div>
    </body>
    </html>
    '''

In [22]:
chat_template_2 = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Meal ChatBot</title>
    <link rel="icon" type="static/uploads/mealplan_logo.png" href="/static/uploads/mealplan_logo.png">
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f0f0f0;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            flex-direction: column;
        }

        .container {
            width: 100%;
            max-width: 800px;
            background-color: #ffffff;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            display: flex;
            flex-direction: column;
            height: 100%;
        }

        h1 {
            text-align: center;
            padding: 20px;
            color: #333;
            margin: 0;
            background-color: #ffcc80;
            color: white;
            border-radius: 10px 10px 0 0;
        }

        .chat-container {
            display: flex;
            flex-direction: column;
            gap: 10px;
            padding: 20px;
            flex-grow: 1;
            overflow-y: auto;
            padding-bottom: 80px;
        }

        .chat-box {
            display: flex;
            flex-direction: column;
            gap: 10px;
        }

        .message {
            padding: 10px;
            border-radius: 5px;
            max-width: 85%;
        }

        .user-message {
            background-color: #d1f7d6;
            align-self: flex-start;
        }

        .bot-message {
            background-color: #f1f1f1;
            align-self: flex-end;
        }

        .chat-form {
            display: flex;
            gap: 10px;
            padding: 20px;
            background-color: #f9f9f9;
            border-top: 1px solid #ddd;
        }

        .chat-form input {
            width: 100%;
            padding: 10px;
            border-radius: 5px;
            border: 1px solid #ccc;
            font-size: 16px;
        }

        .chat-form button {
            padding: 10px 20px;
            border-radius: 5px;
            border: 1px solid #ccc;
            background-color: #ffcc80;
            color: white;
            cursor: pointer;
            font-size: 16px;
        }

        .chat-form button:hover {
            background-color: #ffb74d;
        }

        .dashboard-button {
            padding: 10px 20px;
            margin: 20px;
            border-radius: 5px;
            border: none;
            background-color: #4CAF50;
            color: white;
            cursor: pointer;
            font-size: 16px;
            text-align: center;
        }

        .dashboard-button:hover {
            background-color: #45a049;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Meal Bot</h1>
        <div class="chat-container">
            <div class="chat-box">
                {% if user_message %}
                    <div class="message user-message">
                        <p>{{ user_message }}</p>
                    </div>
                {% endif %}

                {% if recipe %}
                    <div class="message bot-message">
                        <p>{{ recipe }}</p>
                    </div>
                {% endif %}
            </div>
        </div>
        <form method="post" class="chat-form">
            <input type="text" id="user_message" name="user_message" placeholder="Type 'Suggest recipes'" required>
            <button type="submit">Send</button>
        </form>
        <form action="/meal-plan" method="get" style="text-align: center;">
            <button type="submit" class="dashboard-button">View Meal Plan</button>
        </form>
    </div>
    {% if redirect_delay %}
    <script>
        setTimeout(function() {
            window.location.href = "/recipes";  // Redirect to the dashboard after the delay
        }, {{ redirect_delay * 1000 }});  // Delay in milliseconds (7 seconds)
    </script>
    {% endif %}
</body>
</html>
'''



#Supporting functions (Back-end)

In [23]:
from gradio_client import Client

client = Client("s5625570/ChatBot")
def query(message = None, system_message = None):
    result = client.predict(
        message=message,
        system_message=system_message,
        max_tokens=4096,
        temperature=0.7,
        top_p=0.95,
        api_name="/chat"
    )
    return result
def get_user_details(user_email):
    # Function to fetch user details and diet preferences from the database
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor.execute("SELECT height, weight, goal, age, budget, allergies FROM user_details WHERE email = ?", (user_email,))
    user_details = cursor.fetchone()
    cursor.execute("SELECT diet, dislike, preference, activity FROM user_diet WHERE email = ?", (user_email,))
    diet = cursor.fetchone()
    conn.close()
    return user_details, diet

# Predefined questions and answers
chat_predefined_answers = {
    "Hello": "Hello. How can I help you?",
    "Hi": "Hello. How can I help you?",
    "Hey": "Hello. How can I help you?",
    "What is your name?": "I am MealBot, your AI meal planner assistant.",
    "How do I use this app?": "You can input your personal details or edit them in your profile,and when creating a new meal plan select your preferences, and I'll suggest meal plans for you.",
    "What meals can I eat on a vegetarian diet?": "I can suggest vegetarian-friendly meals based on your preferences. Just create a meal plan with the vegetarian diet and type in the chat 'Suggest Recipes' to create a vegetarian-friendly plan .",
    "What is a balanced diet?": "A balanced diet includes a variety of foods to provide necessary nutrients: proteins, carbohydrates, fats, vitamins, and minerals. I'll help you choose meals that meet these criteria. Just type in the chat 'Suggest Recipes'",
    "What should I eat to lose weight?": "To lose weight, a balanced diet with a calorie deficit is essential. I can suggest meals that fit your weight loss goal. Just type in the chat 'Suggest Recipes'",
    "Dinner ideas": "Just type in the chat 'Suggest Recipes', and I'll provide you some of the best meal ideas",
    "Lunch ideas": "Just type in the chat 'Suggest Recipes', and I'll provide you some of the best meal ideas",
    "Breakfast ideas": "Just type in the chat 'Suggest Recipes', and I'll provide you some of the best meal ideas",
    "Can you help me with a meal plan for the week?": "Of course! Just type in the chat 'Suggest Recipes', and I'll create a meal plan for the week.",
    "How do I change my meal plan": "To change your meal plan, edit your details in 'Profile', then in 'Meal Plan' select 'Create a new meal plan'. Choose your options, and finally type 'Suggest recipes' in the meal bot chat to generate a new meal plan",
    "change my meal plan": "To change your meal plan, edit your details in 'Profile', then in 'Meal Plan' select 'Create a new meal plan'. Choose your options, and finally type 'Suggest recipes' in the meal bot chat to generate a new meal plan",
    "change meal plan": "To change your meal plan, edit your details in 'Profile', then in 'Meal Plan' select 'Create a new meal plan'. Choose your options, and finally type 'Suggest recipes' in the meal bot chat to generate a new meal plan",
    "Do you have gluten-free meal options?": "Yes! I can suggest gluten-free meal options. Just let me know if you have any specific gluten-free preferences in the allergies section of your profile.",
    "What is a low-carb diet?": "A low-carb diet reduces carbohydrate intake, focusing on proteins and fats. Let me know if you'd like low-carb meal suggestions.",
    "What are some meal options for a high-protein diet?": "A high-protein diet includes meals rich in proteins like chicken, fish, beans, and tofu. Let me know your preferences when creating a meal plan, and I'll suggest meals.",
    "How do I add my dietary restrictions?": "You can add your dietary restrictions when setting up your profile, or just let me know now, and I’ll consider them in your meal plans.",
    "Can I eat snacks between meals?": "Yes, I can suggest healthy snacks to keep you energized between meals.",
    "What should I eat to gain weight?": "To gain weight, you'll need a calorie surplus with nutrient-dense foods. I can suggest meals tailored to your weight gain goal.",
    "How many meals should I eat per day?": "It depends on your lifestyle and goal. Generally, 3-5 meals a day is common. I can help you plan your meals accordingly.",
    "Can you provide recipes for my meal plan?": "Absolutely! I can suggest recipes based on your preferences, dietary restrictions, and goals. Type 'Suggest recipes'",
    "What is intermittent fasting?": "Intermittent fasting is an eating pattern where you alternate between eating and fasting periods. Let me know if you’d like meal plans based on this method.",
    "Can you suggest meals for a diabetic diet?": "Yes, I can provide meal suggestions suitable for a diabetic diet. Just let me know if you have any other specific preferences.",
    "What is the best meal plan for weight loss?": "A weight loss meal plan typically includes nutrient-dense, lower-calorie foods with a balance of protein, fats, and carbs. I can suggest one based on your needs.",
    "What are the benefits of eating fruits and vegetables?": "Fruits and vegetables are rich in vitamins, minerals, and fiber. They help maintain a healthy weight, improve digestion, and boost immunity.",
    "Can I get meal suggestions based on my calorie goal?": "Yes, I can suggest meals that align with your target calorie intake. Just tell me your calorie goal, and I’ll help you out!",
    "Can you suggest some healthy snacks?": "Sure! Healthy snacks could include fruits, nuts, yogurt, or veggie sticks with hummus. Let me know if you want a list of options."
}

def meal_suggestions(user_email, user_message):
    # Retrieve user details and diet preferences
    user_details, diet = get_user_details(user_email)
    if not user_details:
        return "User details not found."

    # Combine user details into context
    context = (
        f"User details: Height: {user_details[0]},Weight: {user_details[1]}, Goal: {user_details[2]}, Age: {user_details[3]}, Budget: {user_details[4]}, Allergies: {user_details[5]} \n"
    )

    # Handle missing diet information with default values
    if diet is None:
        diet = ("N/A", "N/A", "N/A", "N/A")  # Default values for missing fields
    else:
        diet = tuple(list(diet) + ["N/A"] * (4 - len(diet)))  # Ensure diet has 4 elements

    #Add the diet preferences to the context
    context += (
        f"Diet preferences: Diet: {diet[0]}, Dislikes: {diet[1]}, "
        f"Preferences: {diet[2]}, Activity: {diet[3]}\n"
    )
    system_message = (""
    "You are MealBot, an AI meal planner and nutrition assistant. Your role is to suggest healthy, tailored recipes "
    "and meal ideas based on the user's preferences, dietary restrictions, and goals. Your primary role is to suggest 21 meals, including breakfast, lunch and dinner, separated by days of the week, and based on the users details and diet preferences."
    "Each meal should have a nutrition value"
    "However, if the user specifies a different number of recipes or meals, you must follow their request."
    "if user diet is vegetarian, do not inlcude meat in the meals. If user diet is vegan, do not include any meat or diary in the meal. If user diet is pescatarian, do include meat in the meals but include fish and seafood"
    "Include their preferences in some meals, and remove their dislikes from the meals."
    "You also answer additional questions about food, nutrition, and healthy lifestyles."
    "Always consider the user's input, their diet preferences and the user details when making suggestions. Always include the prices for each meal, the total should be lower then their budget"
)
    # Return predefined answers if the one of the predfined questions was asked
    if any(user_message.strip().lower().startswith(key.lower()) for key in chat_predefined_answers):
       matched_key = next(key for key in chat_predefined_answers if user_message.strip().lower().startswith(key.lower()))
       return chat_predefined_answers[matched_key]

    # Query the AI assistant with the constructed context
    output = query(message=context, system_message=system_message)
    return output

Loaded as API: https://s5625570-chatbot.hf.space ✔


In [24]:
def get_latest_chatbot_response(user_email):
    try:
        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()

        cursor.execute('''
            SELECT response
            FROM chatbot_responses
            WHERE email = ?
            LIMIT 1
        ''', (user_email,))

        result = cursor.fetchone()
        return result[0] if result else None
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None
    finally:
        conn.close()

def save_chatbot_response(user_email, chatbot_response):
    try:
        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()

        # Check if a chatbot response already exists for the user
        cursor.execute('''
            SELECT 1
            FROM chatbot_responses
            WHERE email = ?
            LIMIT 1
        ''', (user_email,))

        existing_response = cursor.fetchone()

        if existing_response:
            # If a response exists, update it
            cursor.execute('''
                UPDATE chatbot_responses
                SET response = ?
                WHERE email = ?
            ''', (chatbot_response, user_email))
        else:
            # If no response exists, insert a new one
            cursor.execute('''
                INSERT INTO chatbot_responses (email, response)
                VALUES (?, ?)
            ''', (user_email, chatbot_response))

        conn.commit()
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        conn.close()

#The App(Back-end)

In [30]:
app = Flask(__name__)
app.secret_key = os.urandom(24)

# Set upload folder inside the static directory
UPLOAD_FOLDER = 'static/uploads'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER


# Database initialization function
def init_db():
    with sqlite3.connect("users.db") as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                email TEXT NOT NULL UNIQUE,
                password TEXT NOT NULL
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS user_details (
                email TEXT PRIMARY KEY,
                height TEXT,
                weight TEXT,
                goal TEXT,
                age TEXT,
                budget TEXT,
                allergies TEXT,
                FOREIGN KEY(email) REFERENCES users(email)
            )
        ''')
        cursor.execute('DROP TABLE IF EXISTS user_diet')
        #Create a new table with the same name and columns
        cursor.execute('''
            CREATE TABLE user_diet (
                email TEXT,
                diet TEXT,
                dislike TEXT,
                preference TEXT,
                activity TEXT,
                FOREIGN KEY(email) REFERENCES users(email)
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS chatbot_responses (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                email TEXT NOT NULL,
                response TEXT NOT NULL,
                FOREIGN KEY(email) REFERENCES user_details(email)
            )
        ''')
    conn.commit()

# Initialize the database
init_db()

# Welcome page
@app.route('/')
def index():
    return render_template_string(welcome_template)


# Registration route
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
      # Enter authentication info
        name = request.form['name']
        email = request.form['email']
        password = request.form['password']
        hashed_password = hash_password(password)

        with sqlite3.connect("users.db") as conn:
            cursor = conn.cursor()
            try:
                cursor.execute(
                    # Save info to database
                    'INSERT INTO users (name, email, password) VALUES (?, ?, ?)',
                    (name, email, hashed_password)
                )
                conn.commit()
                return redirect(url_for('login'))
                # Error message if there is an account with same email in the database
            except sqlite3.IntegrityError:
              error = "Email already exists!"
              return render_template_string(register_template_error, error = error)
    return render_template_string(register_template)

# Login route
@app.route('/login', methods=['GET', 'POST'])
def login():
    try:
        if request.method == 'POST':
            email = request.form['email']
            password = request.form['password']

            with sqlite3.connect("users.db") as conn:
                cursor = conn.cursor()
                cursor.execute('SELECT password FROM users WHERE email = ?', (email,))
                user = cursor.fetchone()

                if user:
                    hashed_password = user[0].decode()  # Decode bytes to string
                    if verify_password(hashed_password, password):
                        session['user_email'] = email

                        # Check if the user already has details
                        cursor.execute('SELECT * FROM user_details WHERE email = ?', (email,))
                        user_details = cursor.fetchone()

                        # Check if there's a chatbot response in the database for this user
                        cursor.execute('''
                            SELECT response
                            FROM chatbot_responses
                            WHERE email = ?
                            LIMIT 1
                        ''', (email,))
                        chatbot_response = cursor.fetchone()

                        # Redirect to the meal plan if chatbot response exists
                        if chatbot_response:
                            return redirect(url_for('meal_plan'))

                        if user_details:
                            # If details exist, go directly to dashboard
                            return redirect(url_for('dashboard'))
                        else:
                            # If no details exist, go to details page
                            return redirect(url_for('details'))
                    else:
                        error = "Invalid email or password!"
                        return render_template_string(login_template_error, error=error)
                else:
                  error = "Email doesn't exist!"
                  return render_template_string(login_template_error, error=error)


    except Exception as e:
        print(f"Error during login: {e}")
        error = "An unexpected error occurred. Please try again later."
        return render_template_string(login_template_error, error=error)

    return render_template_string(login_template)


# Details route
@app.route('/details', methods=['GET', 'POST'])
def details():
    if not session.get('user_email'):
        app.logger.info("User not logged in, redirecting to login.")
        return redirect(url_for('login'))

    if request.method == 'POST':
        try:
            # Retrieve form data
            height = request.form.get('height')
            weight = request.form.get('weight')
            goal = request.form.get('goal')
            age = request.form.get('age')
            budget = request.form.get('budget')
            allergies = request.form.get('allergies')

            # Retrieve user email from session
            user_email = session.get('user_email')
            app.logger.info(f"Updating details for: {user_email}")

            # Save details to the database
            with sqlite3.connect("users.db") as conn:
                cursor = conn.cursor()
                cursor.execute('''
                    INSERT OR REPLACE INTO user_details (email, height, weight, goal, age, budget, allergies)
                    VALUES (?, ?, ?, ?, ?, ?, ?)
                ''', (user_email, height, weight, goal, age, budget, allergies))
                conn.commit()

            # Redirect after successful submission
            return redirect(url_for('dashboard'))
        except Exception as e:
            app.logger.error(f"Error in details route: {e}", exc_info=True)
            return "An error occurred. Please try again.", 500

    return render_template_string(details_template, email=session.get('user_email'))


# Dashboard route
@app.route('/dashboard')
def dashboard():
    user_email = session.get('user_email')
    if not user_email:
        return redirect(url_for('login'))
    return render_template_string(dashboard_template, email=user_email)

# Navigation Bar - Navigation Bar
@app.route('/groceries', methods=['GET', 'POST'])
def groceries():
    ingredients = [
        "Apples", "Bananas", "Carrots", "Tomatoes", "Potatoes", "Onions", "Garlic", "Chicken", "Beef", "Pork",
        "Milk", "Eggs", "Cheese", "Butter", "Bread", "Rice", "Pasta", "Flour", "Sugar", "Salt", "Pepper",
        "Olive Oil", "Cereal", "Yogurt", "Spinach", "Lettuce", "Cucumber", "Bell Peppers", "Broccoli", "Cauliflower",
        "Mushrooms", "Zucchini", "Celery", "Oranges", "Grapes", "Blueberries", "Strawberries", "Pineapple", "Melon",
        "Shrimp", "Fish", "Tofu", "Beans", "Chickpeas", "Lentils", "Corn", "Peas", "Honey", "Chocolate", "Coffee",
        "Tea", "Oats", "Almonds", "Cashews", "Walnuts", "Peanuts", "Peanut Butter", "Coconut Milk", "Avocado",
        "Sweet Potatoes", "Kale", "Swiss Chard", "Green Beans", "Asparagus", "Raspberries", "Blackberries", "Watermelon",
        "Peaches", "Plums", "Kiwi", "Mango", "Papaya", "Cabbage", "Eggplant", "Radishes", "Parsnips", "Beets",
        "Sausage", "Bacon", "Turkey", "Duck", "Salmon", "Tuna", "Cod", "Tilapia", "Halibut", "Crab",
        "Clams", "Scallops", "Lobster", "Quinoa", "Couscous", "Barley", "Maple Syrup", "Vanilla Extract", "Cinnamon",
        "Basil", "Parsley", "Cilantro", "Thyme", "Rosemary", "Ginger", "Turmeric", "Chili Powder", "Paprika"
    ]

    # Initialize variables
    selected_ingredients = session.get('selected_ingredients', [])
    remaining_ingredients = [i for i in ingredients if i not in selected_ingredients]

    if request.method == 'POST':
        # Handle search query
        search_query = request.form.get('search', '').lower()
        if search_query:
            remaining_ingredients = [i for i in ingredients if search_query in i.lower() and i not in selected_ingredients]

        # Handle selected groceries
        selected_ingredients = request.form.getlist('groceries')
        session['selected_ingredients'] = selected_ingredients
        remaining_ingredients = [i for i in ingredients if i not in selected_ingredients]
        message = "Groceries saved!"

        # Render with updated selection and notification
        return render_template_string(
            groceries_template,
            ingredients=ingredients,
            selected_ingredients=selected_ingredients,
            remaining_ingredients=remaining_ingredients,
            message=message
        )

    return render_template_string(
        groceries_template,
        ingredients=ingredients,
        selected_ingredients=selected_ingredients,
        remaining_ingredients=remaining_ingredients,
        message=None
    )
#Chat route - Navigation Bar
@app.route('/chatbot', methods=['GET', 'POST'])
def chatbot():
    if request.method == 'POST':
        user_message = request.form.get('user_message', '').strip()
        user_email = session.get('user_email')
        if not user_email:
            return jsonify({"error": "User email not found in session"}), 400

        # Retrieve user details and preferences from the database
        user_details, diet = get_user_details(user_email)

        if not user_details:
            return jsonify({"error": "User not found"}), 404

       # Check if the user message is a predefined question
        if any(user_message.strip().lower().startswith(key.lower()) for key in chat_predefined_answers):
            matched_key = next(key for key in chat_predefined_answers if user_message.strip().lower().startswith(key.lower()))
            return render_template_string(chat_template_2, user_message=user_message, recipe=chat_predefined_answers[matched_key])

        # Build the user context
        context = f"User details: Height: {user_details[0]}, Weight: {user_details[1]}, Goal: {user_details[2]}, Age: {user_details[3]}, Budget: {user_details[4]}, Allergies: {user_details[5]}\n"
        # Handle missing diet information with default values
        if diet is None:
            diet = ("N/A", "N/A", "N/A", "N/A")
        else:
            diet = tuple(list(diet) + ["N/A"] * (4 - len(diet)))  # Ensure diet has 4 elements
        context += f"Diet preferences: Diet: {diet[0]}, Dislikes: {diet[1]}, Preferences: {diet[2]}, Activity: {diet[3]}\n"
        chatbot_response = meal_suggestions(user_email,user_message)
        save_chatbot_response(user_email, chatbot_response)
        session['chatbot_response'] = chatbot_response
        return render_template_string(chat_template_2, user_message=user_message, recipe=chatbot_response)

    return render_template_string(chat_template)

# Profile route - Navigation Bar
@app.route('/profile', methods=['GET', 'POST'])
def profile():
    user_email = session.get('user_email')  # Ensure user_email is stored in the session

    if not user_email:
        print("User email not found in session. Redirecting to login.")
        return redirect(url_for('login'))  # Redirect to login if no user is logged in
    # Default values
    name, age, weight, height, goal, budget, allergies = "", "", "", "", "", "", ""

    back_link = url_for('dashboard')

    try:
        with sqlite3.connect("users.db") as conn:
            cursor = conn.cursor()

            # Check if there's a chatbot response in the database for this user
            cursor.execute('''
                SELECT response
                FROM chatbot_responses
                WHERE email = ?
                LIMIT 1
            ''', (user_email,))
            chatbot_response = cursor.fetchone()

            # Redirect to the meal plan if chatbot response exists
            if chatbot_response:
                back_link = url_for('meal_plan')

    except Exception as e:
        print(f"Error determining back_link: {e.__class__.__name__}: {e}")

    if request.method == 'POST':
        # Save updated details into the database
        name = request.form['name']
        age = request.form['age']
        weight = request.form['weight']
        height = request.form['height']
        goal = request.form['goal']
        budget = request.form['budget']
        allergies = request.form['allergies']

        try:
            with sqlite3.connect("users.db") as conn:
                cursor = conn.cursor()


                # Update `name` in the `users` table
                cursor.execute('''
                    UPDATE users
                    SET name = ?
                    WHERE email = ?
                ''', (name, user_email))

                # Check if the user exists in the `user_details` table
                cursor.execute('SELECT email FROM user_details WHERE email = ?', (user_email,))
                user_exists = cursor.fetchone()

                if user_exists:
                    # Update existing user details
                    cursor.execute('''
                        UPDATE user_details
                        SET age = ?, weight = ?, height = ?, goal = ?, budget = ?, allergies = ?
                        WHERE email = ?
                    ''', (age, weight, height, goal, budget, allergies, user_email))
                else:
                    # Insert new user details
                    cursor.execute('''
                        INSERT INTO user_details (email, age, weight, height, goal, budget, allergies)
                        VALUES (?, ?, ?, ?, ?, ?, ?)
                    ''', (user_email, age, weight, height, goal, budget, allergies))

                conn.commit()

        except Exception as e:
            print(f"Error updating profile: {e.__class__.__name__}: {e}")
            return render_template_string(
                profile_template,
                name=name,
                age=age,
                weight=weight,
                height=height,
                goal=goal,
                budget=budget,
                allergies=allergies,
                back_link=back_link,
                error="An error occurred while saving your profile. Please try again."
            )
    try:
        with sqlite3.connect("users.db") as conn:
            cursor = conn.cursor()

            # Fetch the name from the `users` table
            cursor.execute('SELECT name FROM users WHERE email = ?', (user_email,))
            user_name = cursor.fetchone()
            name = user_name[0] if user_name else ""

            # Fetch additional profile details from the `user_details` table
            cursor.execute('SELECT age, weight, height, goal, budget, allergies FROM user_details WHERE email = ?', (user_email,))
            user_details = cursor.fetchone()

            if user_details:
                age, weight, height, goal, budget, allergies = user_details
    except Exception as e:
        print(f"Error fetching profile data: {e.__class__.__name__}: {e}")
        return render_template_string(
            profile_template,
            name=name,
            age=age,
            weight=weight,
            height=height,
            goal=goal,
            budget=budget,
            allergies=allergies,
            back_link=back_link,
            error="An error occurred while fetching your profile data. Please try again."
        )
    return render_template_string(
        profile_template,
        name=name,
        age=age,
        weight=weight,
        height=height,
        goal=goal,
        budget=budget,
        allergies=allergies,
        back_link=back_link,
        error=""
    )

# Diet choice route
@app.route('/diet', methods=['GET', 'POST'])
def diet():
    if not session.get('user_email'):
        return redirect(url_for('login'))
    if request.method == 'POST':
        diet = request.form.get('diet')
        user_email = session.get('user_email')
        with sqlite3.connect("users.db") as conn:
            cursor = conn.cursor()
            # Add user choice to the database
            cursor.execute("INSERT INTO user_diet (email, diet) VALUES (?, ?)", (user_email, diet))
            conn.commit()
        return redirect(url_for('activity'))
    return render_template_string(diet_form_template)

# Activity level route
@app.route('/activity', methods=['GET', 'POST'])
def activity():
    if not session.get('user_email'):
        return redirect(url_for('login'))
    if request.method == 'POST':
        activity = request.form.get('activity')
        user_email = session.get('user_email')
        with sqlite3.connect("users.db") as conn:
            cursor = conn.cursor()
            cursor.execute("INSERT INTO user_diet (email, activity) VALUES (?, ?)", (user_email, activity))
            conn.commit()
        return redirect(url_for('dislikes'))
    return render_template_string(activity_form_template)

# Dislikes route
@app.route('/dislikes', methods=['GET', 'POST'])
def dislikes():
    if request.method == 'POST':
        # Retrieve the selected dislikes from the checkboxes
        selected_dislikes = request.form.getlist('dislikes')  # a list of selected checkbox items
        # Retrieve the custom food entered by the user
        custom_dislike = request.form.get('custom_dislike', '').strip()  # Strip any extra spaces

        if custom_dislike:
            selected_dislikes.append(custom_dislike)
        user_email = session.get('user_email')
        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()
        for dislike in selected_dislikes:
            cursor.execute("INSERT INTO user_diet (email, dislike) VALUES (?, ?)", (user_email, dislike))
        conn.commit()

        return redirect(url_for('preferences'))

    return render_template_string(dislikes_form_template)

# Preferences route
@app.route('/preferences', methods=['GET', 'POST'])
def preferences():
    if request.method == 'POST':
        # Retrieve the selected preferences from the checkboxes
        selected_preferences = request.form.getlist('preferences')  # a list of selected checkbox items
        # Retrieve the custom preference entered by the user
        custom_preference = request.form.get('custom_preference', '').strip()  # Strip any extra spaces

        if custom_preference:
            selected_preferences.append(custom_preference)
        user_email = session.get('user_email')
        conn = sqlite3.connect('users.db')  # Connect to your SQLite database
        cursor = conn.cursor()

        for preference in selected_preferences:
            cursor.execute("INSERT INTO user_diet (email, preference) VALUES (?, ?)", (user_email, preference))
        conn.commit()
        return redirect(url_for('chat'))
    return render_template_string(preferences_form_template)

# Chat route
@app.route('/chat', methods=['GET', 'POST'])
def chat():
    if request.method == 'POST':
        user_message = request.form.get('user_message', '').strip()
        user_email = session.get('user_email')
        if not user_email:
            return jsonify({"error": "User email not found in session"}), 400

        # Retrieve user details and preferences from the database
        user_details, diet = get_user_details(user_email)

        if not user_details:
            return jsonify({"error": "User not found"}), 404

        # Check if the user message is a predefined question
        if any(user_message.strip().lower().startswith(key.lower()) for key in chat_predefined_answers):
            matched_key = next(key for key in chat_predefined_answers if user_message.strip().lower().startswith(key.lower()))
            return render_template_string(chat_template_2, user_message=user_message, recipe=chat_predefined_answers[matched_key])

        # Build the user context
        context = f"User details: Height: {user_details[0]}, Weight: {user_details[1]}, Goal: {user_details[2]}, Age: {user_details[3]}, Budget: {user_details[4]}, Allergies: {user_details[5]}\n"
        # Handle missing diet information with default values
        if diet is None:
            diet = ("N/A", "N/A", "N/A", "N/A")
        else:
            diet = tuple(list(diet) + ["N/A"] * (4 - len(diet)))  # Ensure diet has 4 elements
        context += f"Diet preferences: Diet: {diet[0]}, Dislikes: {diet[1]}, Preferences: {diet[2]}, Activity: {diet[3]}\n"
        # Query the meal suggestions with user context
        chatbot_response = meal_suggestions(user_email, user_message)
        save_chatbot_response(user_email, chatbot_response)
        session['chatbot_response'] = chatbot_response
        return render_template_string(chat_template_2, user_message=user_message, recipe=chatbot_response)

    default_message = "Welcome!Use Meal Bot to generate a meal plan made for you."
    session['chatbot_response'] = default_message  # Set the first message
    return render_template_string(chat_template, chatbot_response=default_message)


# Meal plan route
@app.route('/meal-plan')
def meal_plan():
    user_email = session.get('user_email')
    if not user_email:
        return redirect(url_for('login'))
    chatbot_response = get_latest_chatbot_response(user_email)
    if not chatbot_response:
        return render_template_string(meal_plan_template_error)

    # Process chatbot response into a structured format
    week_plan = {}
    days = chatbot_response.strip().split("\n\n")
    for day in days:
        lines = day.split("\n")
        day_name = lines[0].strip(":")
        meals = lines[1:]
        week_plan[day_name] = meals
    return render_template_string(meal_plan_template, week_plan=week_plan)

# Logout route
@app.route('/logout')
def logout():
    session.pop('user_email', None)
    return redirect(url_for('login'))

# Main entry point of the application: this block runs when the script is executed directly
if __name__ == '__main__':
    # Start ngrok and store the public URL
    public_url = start_ngrok()
    # Print the public URL to access the web application
    print(f" * Access the web app at: {public_url}")
    # Run the Flask app on port 5000
    app.run(port=5000)


 * ngrok tunnel URL: https://e7eb-34-41-213-24.ngrok-free.app
 * Access the web app at: https://e7eb-34-41-213-24.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 18:55:40] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 18:55:40] "GET /static/uploads/mealplan_logo.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 18:55:41] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 18:55:42] "GET /static/uploads/mealplan_logo.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 18:55:46] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 18:55:46] "GET /meal-plan HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 18:55:48] "GET /profile HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 18:55:49] "GET /meal-plan HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Dec/2024 18:55:52] "GET /logout HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 